In [ ]:
# Import the libraries relevant to data cleaning and analysis.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
%matplotlib inline

In [ ]:
# Open as a dataframe the file named tmdb.movies.csv that has details such a movie's 
# popularity and vote count.
movie_rating_df = pd.read_csv("unzippedData/tmdb.movies.csv")
movie_rating_df = movie_rating_df.drop(columns='Unnamed: 0', axis=1)

# Print the first 5 values
movie_rating_df.head()

In [ ]:
# Examine the shape (rows by columns) of the dataframe
movie_rating_df.shape

In [ ]:
#  Examine the schema and data types
movie_rating_df.info()

In [ ]:
 # Convert release_date from string data type to datetype
movie_rating_df['release_date'] = pd.to_datetime(movie_rating_df['release_date'])
movie_rating_df.info()

In [ ]:
# Check whether there are missing values
movie_rating_df.isna().sum()

In [ ]:
# Check for duplicate rows in movie_rating_df
# Use keep='first' clause to avoid counting the first value in a set of repeated rows

movie_rating_df.duplicated(keep='first').value_counts()

In [ ]:
# Drop duplicates, keep the first row

movie_rating_df.drop_duplicates(keep='first', inplace=True)

In [ ]:
# Confirm whether duplicates have been removed
movie_rating_df.duplicated(keep='first').value_counts()

Code from student.ipynb

## DATA UNDERSTANDING FOR BOM.MOVIE_GROSS.CSV

In [ ]:
# Open as a dataframe the file named bom.movie_gross that has details such
# a movie's domestic gross income and foreign gross income
movie_gross_df = pd.read_csv('unzippedData/bom.movie_gross.csv')

# Print the first 5 rows
movie_gross_df.head()

In [ ]:
# Examine the last 5 rows
movie_gross_df.tail()

In [ ]:
# Examine the shape (rows by columns) of the dataframe
movie_gross_df.shape

In [ ]:
# Examine the schema and data types
movie_gross_df.info()

In [ ]:
# Examine the missing values
movie_gross_df.isna().sum()

In [ ]:
# Check whether there are duplicated rows
movie_gross_df.duplicated().sum()

In [ ]:
# Examine the nature of the column values
movie_gross_df.value_counts().sum()

## DATA UNDERSTANDING FOR TN.MOVIE_BUDGETS.CSV

In [ ]:
# Open as a dataframe the file named bom.movie_gross that has details such
# a movie's domestic gross income and foreign gross income
# The column id doesn't represent an index, so we won't assign it the index column
movie_budgets_df = pd.read_csv('unzippedData/tn.movie_budgets.csv')
movie_budgets_df.head()

In [ ]:
# Examine the last 5 rows
movie_budgets_df.tail()

In [ ]:
# Examine the shape (rows by columns) of the dataframe
movie_budgets_df.shape

In [ ]:
# Examine the schema and data types
movie_budgets_df.info()

In [ ]:
# Convert release_date from string data type to datetype
movie_budgets_df['release_date'] = pd.to_datetime(movie_budgets_df['release_date'])
movie_budgets_df.info()

In [ ]:
movie_budgets_df.head()

In [ ]:
# Examine missing values in movie_basics_df
movie_budgets_df.isna().sum()

In [ ]:
# Check whether there are duplicated rows
movie_budgets_df.duplicated().sum()

## DATA CLEANING OF MOVIE_GROSS_DF
Cleaning this data set will first involve examining the kind of values in the studio column, as well as the percent of missing values, which will help in deciding how to handle the missing values.

For the missing values in the domestic_gross and foreign_gross columns, an observation of the two datasets movie_gross_df and movie_budget_df shows that I will need to join the two tables at the rows where the names of movies match. This will enable me to analyse the budget, domestic revenue, worldwide revenue, and gross profitability of each movie. 

I will therefore, decide whether to replace the missing values or whole columns of domestic_gross and foreign_gross in movie_gross_df with the matching values from the dataframe movie_budgets_df, after joining the two dataframes.

In [ ]:
# Examine the nature of values in studio column
movie_gross_df['studio'].value_counts()

In [ ]:
# Find percent of missing values in genres column
missing_studios = (movie_gross_df['studio'].isna().sum())/ (len(movie_gross_df['studio']))
print(f"Percent of missing values in studios column is {missing_studios:.2%}")

The values in studio column are categorical and since only 5 (0.15%) are missing, I will replace the missing values with a generic category as a placeholder.

In [ ]:
movie_gross_df['studio'] = movie_gross_df['studio'].fillna('Generic')

In [ ]:
# Confirm that there are no missing values in studio column
movie_gross_df['studio'].isna().sum()

## DATA CLEANING OF MOVIE_BUDGETS_DF
Cleaning this dataset will involve stripping the values in production_budget, domestic_gross, and worldwide_gross columns of the dollar ($) sign and converting them to the float datatype.

In [ ]:
# Use .replace() method to remove all occurences of , and $ signs
# Convert the values to numeric (integer) type
movie_budgets_df['production_budget'] = movie_budgets_df['production_budget'].str.replace(',', '').str.replace('$', '').apply(pd.to_numeric)
movie_budgets_df['domestic_gross'] = movie_budgets_df['domestic_gross'].str.replace(',', '').str.replace('$', '').apply(pd.to_numeric)
movie_budgets_df['worldwide_gross'] = movie_budgets_df['worldwide_gross'].str.replace(',', '').str.replace('$', '').apply(pd.to_numeric)

In [ ]:
# Confirm the data types of movie_budgets_df have changed
movie_budgets_df.info()

## DATA ANALYSIS OF MOVIE_GROSS_DF AND MOVIE_BUDGETS_DF

In [ ]:
# Since we need to combine the dateframes at the rows where movie names are common,
# we will use inner merge instead of inner join.

# Inner merge of movie_gross_df and movie_budgets_df
merged_gross_and_budgets_df = movie_gross_df.merge(movie_budgets_df, left_on='title', 
                                                   right_on='movie', how='inner')
merged_gross_and_budgets_df


In [ ]:
# Drop the id, and movie columns
merged_gross_and_budgets_df.drop(labels=['id', 'movie'], axis=1, inplace=True)

In [ ]:
# Check for missing values in 'domestic_gross_y' and 'worldwide_gross' columns
merged_gross_and_budgets_df[['domestic_gross_y', 'worldwide_gross']].isna().sum()

In [ ]:
# Sampling the dataset to compare the different gross revenue columns

# Create a new dataframe without missing values in 'domestic_gross_x' 
# and 'foreign_gross' columns
sampling_df = merged_gross_and_budgets_df.dropna(subset=['domestic_gross_x', 'foreign_gross'], axis=0).copy()

# Create a new column for sum of 'domestic_gross_x' and 'foreign_gross'
sampling_df['total_gross_x'] = (sampling_df['domestic_gross_x'].apply(pd.to_numeric) + 
                               sampling_df['foreign_gross'].str.replace(',', '').apply(pd.to_numeric))

# Perform random sampling of rows to compare domestic, foreign, and gross revenues
sampling_df = sampling_df[sampling_df["foreign_gross"].notna()].sample(5, random_state=2)
sampling_df

Samples of the sampling_df show that most values of the worldwide_gross column are closely comparable with the corresponding total_gross_x values found by adding domestic_gross_x and foreign_gross. We will therefore drop the domestic_gross_x and foreign_gross in the merged dataframe.

In [ ]:
merged_gross_and_budgets_df.drop(labels=['domestic_gross_x', 'foreign_gross'], axis=1, inplace=True)

Next, since we want analyse profit distribution by studio, we will first find the profits by subtracting production budget from the world_wide gross in the marged dataframe.

In [ ]:
# Create a profits column
merged_gross_and_budgets_df['profits'] = (merged_gross_and_budgets_df['worldwide_gross'] - 
                                          merged_gross_and_budgets_df['production_budget'])
merged_gross_and_budgets_df

In [ ]:
# Group by studio and sort in descending order
grouped_df = merged_gross_and_budgets_df.groupby('studio').sum([['production_budget'], ['profits']])
grouped_and_sorted_df = grouped_df.sort_values(by='profits', ascending=False)
grouped_and_sorted_df.reset_index(inplace=True)
grouped_and_sorted_df

In [ ]:
# Plot a bar chart of gross profit per studio

labels= grouped_and_sorted_df['studio']
values= grouped_and_sorted_df['profits']

plt.figure(figsize=(20, 10))
plt.xticks(rotation=90, fontsize=10, fontweight= 'bold')
plt.title('Profit Distribution of Major Studios in the Movie Industry', fontsize=16, fontweight='bold')
plt.xlabel('Studio', fontsize=10, fontweight='bold')
plt.ylabel('Profit', fontsize=10, fontweight='bold')

# Create the color palette
palette = sns.color_palette("Paired")

# Create the bar plot
plt.bar(labels, values, color= palette, label='Studio')

plt.legend()
plt.show()

In [ ]:
import requests                 # Simpler HTTP requests 
from bs4 import BeautifulSoup   # Python package for pulling data out of HTML and XML files
import pandas as pd             # Python package for data manipulation and analysis
import re                       # regular expressions
from datetime import datetime   # python package to retireve DateTime

In [ ]:
url = 'https://www.imdb.com/chart/top'              # IMDb Top 250 list link
url_text = requests.get(url).text                    # Get the session text for the link
url_soup = BeautifulSoup(url_text, 'html.parser')   # Get data from the HTML

In [ ]:
template = 'https://www.imdb.com%s'

# Get the title links for all the pages
title_links = [template % a.attrs.get('href') for a in url_soup.select('td.titleColumn a')]

imdb_movie_list = []
# Getting the various fields and creating a list of objects with details
#   - ranking | movie_name | url | year | rating | vote_count | summary | production | director | writer_1 | writer_2
#   - genre_1 | genre_2 | genre_3 | genre_4 | release date | censor_rating | movie_length | country | language
#   - budget | gross_worldwide | gross_usa | opening_week_usa

for i in list(range(0, len(title_links))):
    page_url = title_links[i]
    page_text = requests.get(page_url).text
    page_soup = BeautifulSoup(page_text, 'html.parser')
    
# Getting the box office details for language, budget, Opening Weekend USA, 
    # Gross income worldwide and USA, and production company
    box_office_details = []
    box_office_dictionary = {'Country':'','Budget':'','Gross USA':'','Cumulative Worldwide Gross':''}
    for details in page_soup.find_all("div",{"class":"txt-block"}):
        detail = details.get_text(strip=True).split(':')
        if detail[0] in box_office_dictionary:
            box_office_details.append(detail)
    
    for detail in box_office_details: 
        if detail[0] in box_office_dictionary: 
            box_office_dictionary.update({detail[0] : detail[1]}) 

    country = box_office_dictionary['Country'].split("|")
    while len(country) < 4: country.append(' ')

    budget = box_office_dictionary['Budget'].split('(')[0]

    gross_usa = box_office_dictionary['Gross USA']
    gross_worldwide = box_office_dictionary['Cumulative Worldwide Gross'].split(' ')[0]
    
    movie_dict = { 'country': country, 'budget': budget, 'gross_worldwide': gross_worldwide,
        'gross_usa': gross_usa }

    imdb_movie_list.append(movie_dict)


In [ ]:
imdb_movie_list

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_box_office_data(url):
    # Send an HTTP request to the URL and get the webpage's content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the movie data
    table = soup.find('table', {'class': 'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'})

    # Initialize lists to store data
    ranks = []
    titles = []
    grosses = []
    years = []
    #budgets = []

    # Loop through each row of the table
    for row in table.find_all('tr')[1:]:  # Skipping the first row as it contains headers
        columns = row.find_all('td')
        
        # Extract data from each column of the row
        rank = columns[0].text.strip()
        title = columns[1].text.strip()
        gross = columns[2].text.strip()
        year = columns[7].text.strip()
        #budget = columns[9].text.strip()
        
        # Append data to respective lists
        ranks.append(rank)
        titles.append(title)
        grosses.append(gross)
        years.append(year)
        #budgets.append(budget)

    # Create a DataFrame to store the scraped data
    data = {
        'Rank': ranks,
        'Title': titles,
        'Worldwide Lifetime Gross': grosses,
        'Year': years,
    } #'Budget': budgets
    df = pd.DataFrame(data)

    return df

if __name__ == "__main__":
    url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'
    df = scrape_box_office_data(url)
    #df
df

In [ ]:
BASE_URL = 'https://www.boxofficemojo.com'

def get_budget(movie_url):
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    budget_element = soup.find('span', text='Budget')
    if budget_element:
        budget = budget_element.find_next('span').text.strip()
        return budget
    return 'N/A'

def scrape_box_office_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'})

    ranks = []
    titles = []
    grosses = []
    years = []
    budgets = []

    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        rank = columns[0].text.strip()
        title_column = columns[1].find('a')
        title = title_column.text.strip()
        movie_url = BASE_URL + title_column['href']  # Get the URL for the movie's individual page
        gross = columns[2].text.strip()
        year = columns[7].text.strip()
        budget = get_budget(movie_url)
        
        ranks.append(rank)
        titles.append(title)
        grosses.append(gross)
        years.append(year)
        budgets.append(budget)

    data = {
        'Rank': ranks,
        'Title': titles,
        'Worldwide Lifetime Gross': grosses,
        'Year': years,
        'Budget': budgets
    }
    df = pd.DataFrame(data)

    return df

if __name__ == "__main__":
    url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'
    df = scrape_box_office_data(url)
    #print(df)
df

In [ ]:
for i in range(200,1000, 200):
    url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW&offset={}.html".format(i)
    html_page = requests.get(url)
    

In [ ]:
BASE_URL = 'https://www.boxofficemojo.com'

def get_budget(movie_url):
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    budget_element = soup.find('span', text='Budget')
    if budget_element:
        budget = budget_element.find_next('span').text.strip()
        return budget
    return 'N/A'

def scrape_box_office_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'})

    ranks = []
    titles = []
    grosses = []
    years = []
    budgets = []

    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        rank = columns[0].text.strip()
        title_column = columns[1].find('a')
        title = title_column.text.strip()
        movie_url = BASE_URL + title_column['href']  # Get the URL for the movie's individual page
        gross = columns[2].text.strip()
        year = columns[7].text.strip()
        budget = get_budget(movie_url)
        
        ranks.append(rank)
        titles.append(title)
        grosses.append(gross)
        years.append(year)
        budgets.append(budget)

    data = {
        'Rank': ranks,
        'Title': titles,
        'Worldwide Lifetime Gross': grosses,
        'Year': years,
        'Budget': budgets
    }
    df_page2_5 = pd.DataFrame(data)

    return df

if __name__ == "__main__":
    for i in range(200,600,200):
        url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW&offset={}.html".format(i)
        df_page2_5 = scrape_box_office_data(url)
    df_all = pd.concat([df_all, df_page2_5], axis=0)
    #print(df)
df_all

In [ ]:
# Retry
BASE_URL = 'https://www.boxofficemojo.com'

def get_budget(movie_url):
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    budget_element = soup.find('span', text='Budget')
    if budget_element:
        budget = budget_element.find_next('span').text.strip()
        return budget
    return 'N/A'

def scrape_box_office_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'})

    ranks = []
    titles = []
    grosses = []
    years = []
    budgets = []

    for row in table.find_all('tr')[1:]:
        columns = row.find_all('td')
        rank = columns[0].text.strip()
        title_column = columns[1].find('a')
        title = title_column.text.strip()
        movie_url = BASE_URL + title_column['href']  # Get the URL for the movie's individual page
        gross = columns[2].text.strip()
        year = columns[7].text.strip()
        budget = get_budget(movie_url)
        
        ranks.append(rank)
        titles.append(title)
        grosses.append(gross)
        years.append(year)
        budgets.append(budget)

    data = {
        'Rank': ranks,
        'Title': titles,
        'Worldwide Lifetime Gross': grosses,
        'Year': years,
        'Budget': budgets
    }

    return data

if __name__ == "__main__":
    for i in range(200,1000,200):
        url = "https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW&offset={}.html".format(i)
        data = scrape_box_office_data(url)
        df_all = pd.DataFrame(data)
    #print(df)
df_all

In [ ]:
BASE_URL = 'https://www.boxofficemojo.com'

# Define a function for getting the budget, which is on
# a movie's individual page
def get_budget(movie_url):
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    budget_element = soup.find('span', text='Budget')
    if budget_element:
        budget = budget_element.find_next('span').text.strip()
        return budget
    return 'N/A'

# Define a function to scrape rank, title, worldwide lifetime gross,
# year, and budget values
def scrape_box_office_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'})

    # Define empty lists for rank, title, gross, year, and budget values
    ranks = []
    titles = []
    grosses = []
    years = []
    budgets = []

    # Iterate over the rows in table tag
    for row in table.find_all('tr')[1:]:
        
        # Get the values in rank, title, worldwide lifetime gross,
        # and year columns in a row
        columns = row.find_all('td')
        rank = columns[0].text.strip()
        title_column = columns[1].find('a')
        title = title_column.text.strip()
        gross = columns[2].text.strip()
        year = columns[7].text.strip()
        
        # Get the URL for the movie's individual page
        movie_url = BASE_URL + title_column['href']  
        budget = get_budget(movie_url)
        
        # Append the respective lists
        ranks.append(rank)
        titles.append(title)
        grosses.append(gross)
        years.append(year)
        budgets.append(budget)

    # Add the lists values to a dictionary named data, each value at the corresponding key
    data = {
        'Rank': ranks,
        'Title': titles,
        'Worldwide Lifetime Gross': grosses,
        'Year': years,
        'Budget': budgets
    }
    
    # Convert the dictionary 
    df = pd.DataFrame(data)

    return df

if __name__ == "__main__":
    url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'
    df = scrape_box_office_data(url)
    #print(df)
df

In [ ]:
# 800 rows
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE_URL = 'https://www.boxofficemojo.com'
RESULTS_PER_PAGE = 200

def get_budget(movie_url):
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    budget_element = soup.find('span', text='Budget')
    if budget_element:
        budget = budget_element.find_next('span').text.strip()
        return budget
    return 'N/A'

def scrape_box_office_data(url):
    ranks = []
    titles = []
    grosses = []
    years = []
    budgets = []

    for offset in range(0, 800, RESULTS_PER_PAGE):
        page_url = url + f'&offset={offset}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'})

        for row in table.find_all('tr')[1:]:
            columns = row.find_all('td')
            rank = columns[0].text.strip()
            title_column = columns[1].find('a')
            title = title_column.text.strip()
            gross = columns[2].text.strip()
            year = columns[7].text.strip()
            
            movie_url = BASE_URL + title_column['href']
            budget = get_budget(movie_url)
            
            ranks.append(rank)
            titles.append(title)
            grosses.append(gross)
            years.append(year)
            budgets.append(budget)

    data = {
        'Rank': ranks,
        'Title': titles,
        'Worldwide Lifetime Gross': grosses,
        'Year': years,
        'Budget': budgets
    }
    df_all_pages = pd.DataFrame(data)

    return df_all_pages

if __name__ == "__main__":
    url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'
    df_all_pages = scrape_box_office_data(url)
df_all_pages

In [ ]:
df_all_pages

In [ ]:
df_all_pages.to_csv('BOM_800_movies.csv')

In [ ]:
# 1000 rows
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set the base URL to be used to complete the the different URLs needed
BASE_URL = 'https://www.boxofficemojo.com'

# The website has 200 results per page
RESULTS_PER_PAGE = 200

# Define a function for scraping the budget, which is on
# a movie's individual page
def get_budget(movie_url):
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    budget_element = soup.find('span', text='Budget')
    if budget_element:
        budget = budget_element.find_next('span').text.strip()
        return budget
    return 'N/A'

# Define a function to scrape rank, title, worldwide lifetime gross,
# year, and budget values
def scrape_box_office_data(url):
    
    # Define empty lists for rank, title, gross, year, and budget values
    ranks = []
    titles = []
    grosses = []
    years = []
    budgets = []

    # Increment an offset parameter by 200, since each page has 200 results
    for offset in range(0, 1000, RESULTS_PER_PAGE):
        page_url = url + f'&offset={offset}'
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table', {'class': 
                                    'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'
                                   })
        # Iterate over the rows in <table></table> tag
        for row in table.find_all('tr')[1:]:
            # Get the values in rank, title, worldwide lifetime gross,
            # and year columns in each row
            columns = row.find_all('td')
            rank = columns[0].text.strip()
            title_column = columns[1].find('a')
            title = title_column.text.strip()
            gross = columns[2].text.strip()
            year = columns[7].text.strip()
            
            # Define the URL for the movie's individual page
            # and scrape the budget data
            movie_url = BASE_URL + title_column['href']
            budget = get_budget(movie_url)
            
            # Append the respective lists
            ranks.append(rank)
            titles.append(title)
            grosses.append(gross)
            years.append(year)
            budgets.append(budget)
            
            #Delay the start of each cycle by 0.1 sec
            time.sleep(0.1)
            
    # Add the lists values to a dictionary named data, each value at the corresponding key
    data = {
        'Rank': ranks,
        'Title': titles,
        'Worldwide Lifetime Gross': grosses,
        'Year': years,
        'Budget': budgets
    }
    # Convert the dictionary into a dataframe 
    df_all_pages = pd.DataFrame(data)
    
    # Return the dataframe
    return df_all_pages

if __name__ == "__main__":
    url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'
    df_all_pages = scrape_box_office_data(url)
df_all_pages

800 rows × 5 columns

In [ ]:
# 1000 rows
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set the initial URL to be scraped
url = 'https://www.boxofficemojo.com/chart/ww_top_lifetime_gross/?area=XWW'

# Set the base URL to be used to complete the different URLs needed
BASE_URL = 'https://www.boxofficemojo.com'

# The website has 200 results per page
RESULTS_PER_PAGE = 200

# Define a function for scraping the budget, which is on
# a movie's individual page
def get_budget(movie_url):
    response = requests.get(movie_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    budget_element = soup.find('span', text='Budget')
    if budget_element:
        budget = budget_element.find_next('span').text.strip()
        return budget
    return 'N/A'
    
# Define empty lists for rank, title, gross, year, and budget values
ranks = []
titles = []
grosses = []
years = []
budgets = []

# Increment an offset parameter by 200, since each page has 200 results
for offset in range(0, 1000, RESULTS_PER_PAGE):
    page_url = url + f'&offset={offset}'
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 
                                'a-bordered a-horizontal-stripes a-size-base a-span12 mojo-body-table mojo-table-annotated'
                               })
    # Iterate over the rows in <table></table> tag
    for row in table.find_all('tr')[1:]:
        # Get the values in rank, title, worldwide lifetime gross,
        # and year columns in each row
        columns = row.find_all('td')
        rank = columns[0].text.strip()
        title_column = columns[1].find('a')
        title = title_column.text.strip()
        gross = columns[2].text.strip()
        year = columns[7].text.strip()

        # Define the URL for the movie's individual page
        # and scrape the budget data
        movie_url = BASE_URL + title_column['href']
        budget = get_budget(movie_url)

        # Append the respective lists
        ranks.append(rank)
        titles.append(title)
        grosses.append(gross)
        years.append(year)
        budgets.append(budget)

        #Delay the start of each cycle by 0.1 sec
        time.sleep(0.1)

# Add the lists values to a dictionary named data, each value at the corresponding key
data = {
    'Rank': ranks,
    'Title': titles,
    'Worldwide Lifetime Gross': grosses,
    'Year': years,
    'Budget': budgets
}
# Convert the data to a dataframe
df_all_pages = pd.DataFrame(data)

# Print the dataframe
df_all_pages

In [ ]:
import pandas as pd

# Create a sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'City': ['New York', 'San Francisco', 'Los Angeles']}

df_index = pd.DataFrame(data)

# Group the DataFrame by 'City' column
grouped_df = df_index.groupby('City').mean()

# Reset the index and move the 'City' column back into the DataFrame
grouped_df.reset_index(drop=False, inplace=True)

# Output the DataFrame with the 'City' column back as a regular column
print(grouped_df)